### Fetch a job description from elastic search

In [1]:
from opensearchpy import OpenSearch, helpers

In [2]:
host = 'search-swift-hire-dev-jfmldmym4cfbiwdhwmtuqq6ihy.us-west-2.es.amazonaws.com'
port = 443
auth = ('swift', 'Hire123!') # For testing only. Don't store credentials in code.

es_client = OpenSearch(
    hosts = [{'host': host, 'port': port}],
    http_compress = True, # enables gzip compression for request bodies
    http_auth = auth,
    use_ssl = True,
    ssl_assert_hostname = False,
    ssl_show_warn = False,
)

In [3]:
ES_JOB_INDEX = 'swifthire_jobs_dev'
response = es_client.search(
    index=ES_JOB_INDEX,
    body={
        "query": {
            "match": {
                "searched_job_title": "Data Scientist"
            }
        }
    }
)

In [4]:
job = response['hits']['hits'][0]

In [5]:
job

{'_index': 'swifthire_jobs_dev',
 '_id': 'y5R9HZABhNg_xFXvWjxh',
 '_score': 2.6568744,
 '_source': {'title': 'Senior Data Scientist - Reinforcement Learning',
  'companyName': 'Home Depot / THD',
  'location': 'Vancouver',
  'via': 'via Glassdoor',
  'description': "Position Purpose:\n\nThe Sr. Data Scientist is responsible for leading data science initiatives that drive business profitability, increased efficiencies and improved customer experience. This role assists in the development of the Home Depot advanced analytics infrastructure that informs decision making. Sr. Data Scientists are expected to seek out business opportunities to leverage data science... as a competitive advantage. Based on the specific data science team, this role would need to be Proficient in one or more data science specializations, such as optimization, computer vision, recommendation, search or NLP.\n\nAs a Sr. Data Scientist, you will serve as a lead on data science projects, collaborating with project/pr

### Use OpenAI to summarize it

In [6]:
openai_api_key = # your api key

In [7]:
import openai



In [8]:
import os
from openai import OpenAI
openai_client = OpenAI(api_key=OPENAI_API_KEY)



In [9]:
from openai import OpenAI

client = OpenAI(
    api_key=OPENAI_API_KEY  
)

models = client.models.list()
print(models)

SyncPage[Model](data=[Model(id='text-embedding-ada-002', created=1671217299, object='model', owned_by='openai-internal'), Model(id='gpt-4o-mini', created=1721172741, object='model', owned_by='system'), Model(id='gpt-4o-mini-2024-07-18', created=1721172717, object='model', owned_by='system'), Model(id='gpt-4o', created=1715367049, object='model', owned_by='system'), Model(id='whisper-1', created=1677532384, object='model', owned_by='openai-internal'), Model(id='gpt-3.5-turbo', created=1677610602, object='model', owned_by='openai'), Model(id='tts-1', created=1681940951, object='model', owned_by='openai-internal')], object='list')


In [10]:
prompt = f"""
Given the following job description and highlights, summarize the key information, return in point form:
- job title:
- company:
- key skills required:
- year of experience required:
- location:
- salary range:

Job Description: {job["_source"]["description"]}
Job Highlights: {job["_source"]["jobHighlights"]}

"""

response = openai_client.chat.completions.create(
    model='gpt-4o-mini-2024-07-18',
    messages=[
        {"role": "user", "content": prompt}
    ]
)

summary = response.choices[0].message.content.strip()

In [11]:
print(summary)

- **Job Title:** Senior Data Scientist  
- **Company:** Home Depot  
- **Key Skills Required:**  
  - Proficiency in data science specializations (optimization, computer vision, recommendation, search, NLP)  
  - Strong experience with Python and deep learning frameworks (TensorFlow, PyTorch)  
  - Experience in designing and implementing Reinforcement Learning (RL) algorithms  
  - Excellent communication and collaboration skills  
  - Understanding of machine learning principles and experimentation  
- **Year of Experience Required:** 5 years  
- **Location:** Not explicitly mentioned, but implies a comfortable indoor area  
- **Salary Range:** Not specified


In [12]:
### update the es object with the summary
response = es_client.update(
    index=ES_JOB_INDEX,
    id=job["_id"],
    body={
        "doc": {
            "ai_summary": summary
        }
    }
)